CONTROLE DE AVISOS

In [12]:
import warnings
#Essa linha serve para ignorar os avisos que aparecem no código
warnings.filterwarnings("ignore")

INSTALAÇÃO DAS BIBLIOTECAS NECESSÁRIAS PARA A APLICAÇÃO

In [ ]:
!python -m pip install crewai

IMPORTAÇÃO DAS BIBLIOTECAS NECESSÁRIAS E DEFINIÇÃO DE VARIÁVEIS DE AMBIENTE

In [14]:
from crewai import Agent, Task, Crew
import os 
from IPython.display import Markdown

#Definindo a chave de API do OpenAI
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
#Definindo o modelo do ChatGPT que será usado
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

CRIAÇÃO DOS AGENTES

In [15]:
#Criação do Agente por meio de uma instanciação da classe Agent
searcher = Agent(
    #Aqui é definido o nome/função do agente
    role="Content Searcher",
    #Aqui é definido o objetivo do agente
    goal="Conduct comprehensive research on the given list of symptoms: {sintomas}, identifying all possible related diseases."
    "Ensure that the findings are insightful, factually accurate, and cover a wide range of potential conditions.",
    #Aqui é definida a "identidade" do agente,
    #ou seja, o modo como ele irá agir
    backstory="You're working on researching possible diseases related to the given symptoms: {sintomas}."
    "You base your research on reliable medical sources and databases, ensuring a comprehensive and factually accurate analysis."
    "You follow the main objectives and direction provided, ensuring that all possible conditions related to the symptoms are explored."
    "You provide objective and impartial insights, always backing them up with medically verified information."
    "You acknowledge when certain correlations between symptoms and diseases are speculative or not fully confirmed by medical consensus.",
    #Aqui é definido que ele não pode delegar tarefas
    allow_delegation=False,
    #Parâmetro que define se as mensagens sobre as 
    #ações realizadas pelos agentes serão exibidas
    verbose=True
)

In [ ]:
#Criação do Agente por meio de uma instanciação da classe Agent
editor = Agent(
    #Aqui é definido o nome/função do agente
    role="Editor",
    #Aqui é definido o objetivo do agente
    goal="Analyze possible diseases and identify the three most probable conditions based on medical data and diagnostic likelihood.",
    #Aqui é definida a "identidade" do agente,
    #ou seja, o modo como ele irá agir
    backstory="You are an editor who receives a list of possible diseases from the Research Agent."
    "Your goal is to review the findings to ensure they are medically relevant, factually accurate, and based on reliable diagnostic criteria."
    "You analyze the probabilities of each condition and refine the list to highlight the three most likely diseases."
    "You ensure that the final selection maintains an objective, data-driven approach, avoiding speculation and unreliable sources.",
    #Aqui é definido que ele não pode delegar tarefas
    allow_delegation=False,
    #Parâmetro que define se as mensagens sobre as 
    #ações realizadas pelos agentes serão exibidas
    verbose=True
)

CRIAÇÃO DAS TASKS

In [ ]:
#Criação da task de pesquisa das doenças relacionadas aos sintomas
search = Task(
    #Aqui é definida a descrição da tarefa, o que esperamos que o agente faça, a tarefa planejada
    #Nesse caso, a tarefa é pesquisar doenças relacionadas aos sintomas
    description=(
        "1. Gather comprehensive information on all possible diseases related to {sintomas}.\n"
        "2. Prioritize medically verified sources and established diagnostic criteria.\n"
        "3. Organize findings into a structured list, ensuring clarity and accuracy.\n"
        "4. Provide references or supporting data to substantiate the research."
    ),
    #Aqui é definido a saída esperada, o que esperamos que o agente entregue
    expected_output="A comprehensive research document listing all possible diseases, including their likelihood, medical references, and supporting data.",
    #Aqui é definido o Agente responsável pela tarefa, que, 
    #nesse caso, será o que irá pesquisar as doenças
    agent=searcher
)

In [ ]:
#Criação da task de edição das doenças mais prováveis
edit = Task(
    #Aqui é definida a descrição da tarefa, o que esperamos que o agente faça, a tarefa planejada
    #Nesse caso, editar e escolher quais são as doenças mais prováveis
    description=(
        "1. Review the list of diseases related to the given symptoms.\n"
        "2. Analyze the medical data and diagnostic criteria for each condition.\n"
        "3. Identify the three most probable diseases based on the research.\n"
        "4. Ensure that the final selection is objective, data-driven, and medically relevant."
    ),
    #Aqui é definido a saída esperada, o que esperamos que o agente entregue
    #Nesse caso, um documento editado com as três doenças mais prováveis
    expected_output="An edited research document highlighting the three most probable diseases related to the symptoms, supported by medical data and diagnostic criteria.",
    #Aqui é definido o Agente responsável pela tarefa, que,
    #nesse caso, será o que irá editar as doenças
    agent=editor
)

CRIAÇÃO DO CREW (GRUPO DOS AGENTES QUE TRABALHAM JUNTOS)

In [19]:
crew = Crew(
    #Definição da lista dos agentes que irão participar do processo
    agents=[searcher, editor],
    #Definição da lista das tarefas que irão participar, 
    #as tarefas serão realizadas na sequência que estão nessa lista
    #ou seja, primeiro o search e depois o edit
    tasks=[search, edit],
    #Parâmetro que define se as mensagens sobre as 
    #ações realizadas pelos agentes serão exibidas
    verbose=True
)

CHAMANDO A FUNÇÃO CREW PARA QUE O SISTEMA SEJA REALIZADO

In [20]:
# Iniciar o processo de pesquisa e edição das doenças relacionadas aos sintomas
result = crew.kickoff(inputs={"sintomas": "fever, cough, headache"})

 
[2025-03-12 22:42:34][🚀 CREW 'CREW' STARTED, 56B5316E-0DCE-47D3-A598-AEFA1D39EE28]: 2025-03-12 22:42:34.010572
 
[2025-03-12 22:42:34][📋 TASK STARTED: 1. GATHER COMPREHENSIVE INFORMATION ON ALL POSSIBLE DISEASES RELATED TO FEVER, COUGH, HEADACHE.
2. PRIORITIZE MEDICALLY VERIFIED SOURCES AND ESTABLISHED DIAGNOSTIC CRITERIA.
3. ORGANIZE FINDINGS INTO A STRUCTURED LIST, ENSURING CLARITY AND ACCURACY.
4. PROVIDE REFERENCES OR SUPPORTING DATA TO SUBSTANTIATE THE RESEARCH.]: 2025-03-12 22:42:34.038483
 
[2025-03-12 22:42:34][🤖 AGENT 'CONTENT SEARCHER' STARTED TASK]: 2025-03-12 22:42:34.044717
# Agent: Content Searcher
## Task: 1. Gather comprehensive information on all possible diseases related to fever, cough, headache.
2. Prioritize medically verified sources and established diagnostic criteria.
3. Organize findings into a structured list, ensuring clarity and accuracy.
4. Provide references or supporting data to substantiate the research.
 
[2025-03-12 22:42:34][🤖 LLM CALL STARTED]: 202

VISUALIZAÇÃO DAS POSSÍVEIS DOENÇAS "FORMATADAS"

In [21]:
Markdown(result.raw)

Based on the symptoms provided (fever, cough, headache), the three most probable diseases are Influenza (Flu), Common Cold, and COVID-19. 

1. **Influenza (Flu)**
- **Symptoms:** Fever, cough, headache, muscle aches, fatigue.
- **Likelihood:** High during flu season.
- **Reference:** Centers for Disease Control and Prevention (CDC).

2. **Common Cold**
- **Symptoms:** Mild fever, cough, headache, runny or stuffy nose.
- **Likelihood:** Common viral infection.
- **Reference:** National Institute of Allergy and Infectious Diseases (NIAID).

3. **COVID-19**
- **Symptoms:** Fever, cough, headache, loss of taste or smell, shortness of breath.
- **Likelihood:** Pandemic status, especially during outbreaks.
- **Reference:** World Health Organization (WHO).

While other conditions on the list may also present with similar symptoms, the prevalence and current global situation make Influenza, Common Cold, and COVID-19 the most likely causes. It is important for individuals experiencing these symptoms to seek medical advice for proper diagnosis and management.

OUTRA DEMONSTRAÇÃO COM MAIS SINOTMAS

In [23]:
sintomas = "Fever, Headache, Fatigue, Nausea, Shortness of Breath, Diarrhea, Congestion"
result = crew.kickoff(inputs={"sintomas": sintomas})

 
[2025-03-12 23:05:06][🚀 CREW 'CREW' STARTED, 56B5316E-0DCE-47D3-A598-AEFA1D39EE28]: 2025-03-12 23:05:06.018243
 
[2025-03-12 23:05:06][📋 TASK STARTED: 1. GATHER COMPREHENSIVE INFORMATION ON ALL POSSIBLE DISEASES RELATED TO FEVER, HEADACHE, FATIGUE, NAUSEA, SHORTNESS OF BREATH, DIARRHEA, CONGESTION.
2. PRIORITIZE MEDICALLY VERIFIED SOURCES AND ESTABLISHED DIAGNOSTIC CRITERIA.
3. ORGANIZE FINDINGS INTO A STRUCTURED LIST, ENSURING CLARITY AND ACCURACY.
4. PROVIDE REFERENCES OR SUPPORTING DATA TO SUBSTANTIATE THE RESEARCH.]: 2025-03-12 23:05:06.043264
 
[2025-03-12 23:05:06][🤖 AGENT 'CONTENT SEARCHER' STARTED TASK]: 2025-03-12 23:05:06.047262
# Agent: Content Searcher
## Task: 1. Gather comprehensive information on all possible diseases related to Fever, Headache, Fatigue, Nausea, Shortness of Breath, Diarrhea, Congestion.
2. Prioritize medically verified sources and established diagnostic criteria.
3. Organize findings into a structured list, ensuring clarity and accuracy.
4. Provide re

In [24]:
Markdown(result.raw)

Based on the presented symptoms and known medical data, the three most probable diseases for the individual experiencing fever, headache, fatigue, nausea, shortness of breath, diarrhea, and congestion are:

1. COVID-19: This viral infection is a significant possibility given its wide range of symptoms that align with those described, including fever, fatigue, headache, nausea, shortness of breath, and congestion. The potential presence of COVID-19 variants further increases the likelihood of this condition.

2. Influenza (flu): Influenza is another probable diagnosis as it commonly manifests with symptoms like fever, headache, fatigue, nausea, and congestion, closely matching the symptoms reported. Shortness of breath can also be a symptom in severe cases of influenza.

3. Pneumonia: Both viral and bacterial pneumonia can present with symptoms such as fever, fatigue, headache, shortness of breath, and in some cases, nausea. While diarrhea is less common, it can still manifest as a symptom, making pneumonia a plausible explanation for the symptoms at hand.

It is crucial to emphasize that a definitive diagnosis can only be made through proper medical evaluation and testing by healthcare professionals. The above analysis is based on the provided symptoms and established medical literature, but individual cases may vary, warranting personalized medical attention for accurate diagnosis and management.